In [ ]:
import pandas as pd
import re
import censusdata

In [ ]:
file_name = '2016_US_County_Level_Presidential_Results.csv'
url = 'https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/%s' % file_name
df_elect_2016_county = pd.read_csv(url)
df_elect_2016_county.rename(columns={"combined_fips": "fips"}, inplace=True)


In [ ]:
cols = []
for d in censusdata.search('acs1', 2016, 'concept', '^sex by age$'):
    print(','.join(d))
    cols.append(d[0])

In [ ]:
df_county = censusdata.download('acs5', 2016, censusdata.censusgeo([('county', '*')]), cols)

In [ ]:
df_county['voting_age_men'] = df_county.iloc[:,6:24].sum(axis=1)
df_county['voting_age_women'] = df_county.iloc[:,30:48].sum(axis=1)

In [74]:
df_county['fips'] = df_county.index.to_series().apply(lambda x: "%05d" % int(x.params()[0][1] + x.params()[1][1]))

In [76]:
df_elect_2016_county['fips'] = df_elect_2016_county['fips'].apply(lambda x: "%05d" % x)

In [79]:
df = pd.merge(
    df_elect_2016_county[['votes_dem', 'votes_gop', 'total_votes', 'state_abbr', 'county_name', 'fips']],
    df_county[['voting_age_men', 'voting_age_women', 'fips']],
    on=['fips'])

In [80]:
df['turnout'] = df['total_votes'] / (df['voting_age_men'] + df['voting_age_women'])

In [81]:
df.to_csv('voter-turnout.csv', index=False)